Để sử dụng Reinforcement Learning (RL) tối ưu giá cho từng loại sản phẩm dựa trên dữ liệu lịch sử, chúng ta sẽ triển khai một giải pháp tập trung vào từng sản phẩm. Quy trình này bao gồm:
	1.	Tiền xử lý dữ liệu: Đọc và chuẩn bị dữ liệu từ file CSV hoặc DataFrame.
	2.	Thiết kế môi trường RL: Mô phỏng môi trường định giá cho từng sản phẩm.
	3.	Huấn luyện RL: Dùng Q-Learning hoặc Deep Q-Learning để tối ưu giá cho mỗi sản phẩm.

Dưới đây là mã minh họa hoàn chỉnh:

1. Tiền xử lý dữ liệu

Giả sử bạn có dữ liệu trong một file CSV với các cột:
	•	product_id: Mã sản phẩm.
	•	date: Ngày bán.
	•	price: Giá bán.
	•	quantity_sold: Số lượng bán ra.
	•	revenue: Doanh thu (có thể tính từ price * quantity_sold nếu thiếu).

import pandas as pd
import numpy as np

# Đọc dữ liệu từ file CSV
data = pd.read_csv('sales_data.csv')

# Kiểm tra dữ liệu
print(data.head())

# Tách dữ liệu theo sản phẩm
products = data['product_id'].unique()
product_data = {p: data[data['product_id'] == p] for p in products}

2. Mô hình hóa môi trường RL cho từng sản phẩm

Tạo môi trường RL cho một sản phẩm dựa trên dữ liệu lịch sử của nó.

class ProductPricingEnvironment:
    def __init__(self, product_data, num_price_bins=10):
        self.data = product_data
        self.num_price_bins = num_price_bins
        self.prices = np.linspace(
            self.data['price'].min(), self.data['price'].max(), num_price_bins
        )
        self.current_step = 0

    def reset(self):
        self.current_step = 0
        return self._get_state()

    def step(self, action):
        price = self.prices[action]
        current_row = self.data.iloc[self.current_step]
        
        # Tính lượng cầu dự đoán và doanh thu
        demand = current_row['quantity_sold']  # Có thể cải tiến dự đoán bằng mô hình hồi quy
        revenue = price * demand

        # Chuyển sang bước tiếp theo
        self.current_step += 1
        done = self.current_step >= len(self.data)

        return self._get_state(), revenue, done

    def _get_state(self):
        if self.current_step < len(self.data):
            return self.data.iloc[self.current_step][['price', 'quantity_sold']].values
        else:
            return np.array([0, 0])

3. Q-Learning để tối ưu giá

from collections import defaultdict

class QLearningAgent:
    def __init__(self, num_actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = defaultdict(lambda: np.zeros(num_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.num_actions = num_actions

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.num_actions)
        return np.argmax(self.q_table[tuple(state)])

    def update(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[tuple(next_state)])
        td_target = reward + self.gamma * self.q_table[tuple(next_state)][best_next_action]
        td_error = td_target - self.q_table[tuple(state)][action]
        self.q_table[tuple(state)][action] += self.alpha * td_error

4. Huấn luyện RL cho từng sản phẩm

# Tham số RL
num_price_bins = 10
num_episodes = 100

optimal_prices = {}

for product_id, product_df in product_data.items():
    print(f"Đang huấn luyện cho sản phẩm {product_id}...")

    # Khởi tạo môi trường và agent
    env = ProductPricingEnvironment(product_df, num_price_bins)
    agent = QLearningAgent(num_actions=num_price_bins)

    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            state = next_state
            total_reward += reward

        print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}")

    # Lưu giá tối ưu cho sản phẩm
    optimal_action = np.argmax(agent.q_table[tuple(env.reset())])
    optimal_price = env.prices[optimal_action]
    optimal_prices[product_id] = optimal_price

print("Giá tối ưu cho từng sản phẩm:")
print(optimal_prices)

5. Mở rộng với Deep Q-Learning

Nếu số lượng sản phẩm hoặc trạng thái quá lớn, bạn có thể thay Q-Table bằng mạng nơ-ron thông qua thư viện như TensorFlow hoặc PyTorch.

6. Kết quả

	•	Mỗi sản phẩm sẽ có một giá tối ưu được xác định từ quá trình huấn luyện RL.
	•	Tùy thuộc vào dữ liệu, RL sẽ học cách cân bằng giữa giá bán và lượng cầu để tối đa hóa doanh thu.

Nếu bạn cần hướng dẫn tích hợp hoặc tinh chỉnh thuật toán để phù hợp với dữ liệu cụ thể, hãy cho mình biết!

In [7]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file CSV
data = pd.read_excel('E:/CHUYÊN ĐỀ/DataBLK1.xlsx')

# Kiểm tra dữ liệu
print(data.head())

# Tách dữ liệu theo sản phẩm
products = data['Dòng sản phẩm'].unique()
product_data = {p: data[data['Dòng sản phẩm'] == p] for p in products}

  Dòng sản phẩm       Ngày   Giá  Số lượng  Doanh thu
0    CONNECTORS 2021-01-04  6500        42     273000
1    CONNECTORS 2021-01-05  6500        35     227500
2    CONNECTORS 2021-01-06  6500        38     247000
3    CONNECTORS 2021-01-07  6500        41     266500
4    CONNECTORS 2021-01-08  6500        46     299000


In [9]:
predict = pd.read_excel('E:/CHUYÊN ĐỀ/predictDT.xlsx')
minimal =pd.read_excel("E:/CHUYÊN ĐỀ/minimal.xlsx")
DT_yeucau = predict['LSTM'].mean()
SL_yeucau = minimal['Số lượng'].mean()
print(DT_yeucau)
print(SL_yeucau)

21650822.6
305.7187516553959


In [3]:
import pandas as pd

# Giả sử bạn đã tải dữ liệu vào DataFrame
# Dữ liệu ban đầu bao gồm: 'date', 'product_line', 'price', 'quantity_sold', 'revenue'
data = pd.read_excel('E:/CHUYÊN ĐỀ/DataBLK1.xlsx')

# Tính giá trung bình có trọng số, tổng số lượng và tổng doanh thu cho từng ngày
new_data = (
    data.groupby('Ngày')
    .apply(lambda x: pd.Series({
        'Mức giá chung': (x['Giá'] * x['Số lượng']).sum() / x['Số lượng'].sum(),  # Giá trung bình có trọng số
        'Tổng số lượng': x['Số lượng'].sum(),  # Tổng số lượng bán ra
        'Tổng doanh thu': x['Doanh thu'].sum()  # Tổng doanh thu
    }))
    .reset_index()
)

# Lưu dữ liệu mới vào file CSV
#summary_data.to_csv('summary_data_weighted.csv', index=False)

print(new_data.head())  # Hiển thị dữ liệu mới

        Ngày  Mức giá chung  Tổng số lượng  Tổng doanh thu
0 2021-01-04   41333.333333          150.0       6200000.0
1 2021-01-05   53130.597015          134.0       7119500.0
2 2021-01-06   43500.000000          146.0       6351000.0
3 2021-01-07   49481.617647          136.0       6729500.0
4 2021-01-08   44477.987421          159.0       7072000.0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10532\2635757210.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [4]:
#Tạo môi trường RL dựa trên dữ liệu lịch sử của công ty.
class ProductPricingEnvironment:
    def __init__(self, product_data,target_revenue, pre_quantity ,num_price_bins=100):
        self.data = product_data
        self.num_price_bins = num_price_bins
        self.target_revenue = DT_yeucau
        self.pre_quantity = SL_yeucau
        self.prices = np.linspace(
            self.data['Mức giá chung'].min(), self.data['Mức giá chung'].max(), num_price_bins
        )
        self.current_step = 0
    def reset(self):
        self.current_step = 0
        return self._get_state()
    def step(self, action):
        price = self.prices[action]
        current_row = self.data.iloc[self.current_step] 
        # Tính lượng cầu dự đoán và doanh thu
        pre_revenue = price * self.pre_quantity
        #Phần thưởng
        reward =-abs(pre_revenue-self.target_revenue)
        # Chuyển sang bước tiếp theo
        self.current_step += 1
        done = self.current_step >= len(self.data)
        return self._get_state(), reward, done
    def _get_state(self):
        if self.current_step < len(self.data):
            return self.data.iloc[self.current_step][['Mức giá chung', 'Tổng số lượng']].values
        else:
            return np.array([0, 0])

In [5]:
#3. Q-Learning để tối ưu giá

from collections import defaultdict

class QLearningAgent:
    def __init__(self, num_actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = defaultdict(lambda: np.zeros(num_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.num_actions = num_actions

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.num_actions)
        return np.argmax(self.q_table[tuple(state)])

    def update(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[tuple(next_state)])
        td_target = reward + self.gamma * self.q_table[tuple(next_state)][best_next_action]
        td_error = td_target - self.q_table[tuple(state)][action]
        self.q_table[tuple(state)][action] += self.alpha * td_error

In [6]:
#4. Huấn luyện RL
import numpy as np
import random

# Cố định seed cho NumPy và random
seed = 42
np.random.seed(seed)
random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)
# Tham số RL
num_price_bins = 100
num_episodes = 1000

optimal_prices = {}

#for product_id, product_df in product_data.items():
    #print(f"Đang huấn luyện cho sản phẩm {product_id}...")

    # Khởi tạo môi trường và agent
env = ProductPricingEnvironment(new_data, DT_yeucau,SL_yeucau,num_price_bins)
agent = QLearningAgent(num_actions=num_price_bins)
initial_epsilon = 0.1
min_epsilon = 0.0  # Không khám phá sau vài vòng lặp
decay_rate = 0.995


for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            state = next_state
            total_reward += reward
        # Giảm epsilon sau mỗi episode
        agent.epsilon = max(min_epsilon, agent.epsilon * decay_rate)

        print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}")

    # Lưu giá tối ưu cho hàm mục tiêu
optimal_action = np.argmax(agent.q_table[tuple(env.reset())])
optimal_price = env.prices[optimal_action]
#optimal_prices[product_id] = optimal_price

print(optimal_price)

Episode 1/1000, Total Reward: -12640683575.694237
Episode 2/1000, Total Reward: -12538242670.968391
Episode 3/1000, Total Reward: -12141527047.457342
Episode 4/1000, Total Reward: -12014773725.656727
Episode 5/1000, Total Reward: -11922924450.584436
Episode 6/1000, Total Reward: -11587926978.244703
Episode 7/1000, Total Reward: -11474151747.510965
Episode 8/1000, Total Reward: -11236039145.731411
Episode 9/1000, Total Reward: -11027642085.663912
Episode 10/1000, Total Reward: -10791709519.908268
Episode 11/1000, Total Reward: -10641778033.057016
Episode 12/1000, Total Reward: -10369871610.053314
Episode 13/1000, Total Reward: -10200883384.058008
Episode 14/1000, Total Reward: -10021330286.34007
Episode 15/1000, Total Reward: -9778450224.555191
Episode 16/1000, Total Reward: -9565218531.56768
Episode 17/1000, Total Reward: -9353866518.369465
Episode 18/1000, Total Reward: -9180966023.267452
Episode 19/1000, Total Reward: -8997299693.874796
Episode 20/1000, Total Reward: -8744691387.5671

In [7]:
pp1 = minimal['Mức giá chung tối ưu'].mean()
pp2 = optimal_price
(pp2-pp1)/pp1*100

0.4795825480089839